# Title: Mapping in R
### Authors: Richard Johansen
### University of Cincinnati Libraries
### 4/18/2019

### REFERENCES:
Code: https://github.com/RAJohansen/UCL_Workshops/tree/master/Mapping_in_R
Data: www.gapminder.org/data/
Geocomputation in R: https://geocompr.robinlovelace.net/ 

### Section I: Introduction to R

#### Step 1: Load Required Packages

In [ ]:
# install.packages(c("tidyverse","gapminder", "maptools","tmap", "marmap", "spData", "spDataLarge", "spDataLarge"))

library(tidyverse)
library(tmap)
library(leaflet)
library(spData)
library(lattice)
library(maptools)
library(gapminder)

#### Step 2: Create an R Object (data frame) from the data _gapminder_

In [ ]:
df <- gapminder

#### Step 3: Explore the dataframe _df_ 

1. How many variables are in the data?
2. How many observations are there in the data?
3. What type of data is this?
4. Do you have expectations at this point?

In [ ]:
str(df)
head(df)
tail(df)

We can create a table to quickly view the number of observations for each continent

In [ ]:
table(df$continent)

Always use the summary tool to explore new data to get a quantitative overview of the data

In [ ]:
summary(df)

#### Step 4: Visualization in R

Using the _plot_ to visualize life expectancy by country

In [ ]:
plot(lifeExp ~ year, gapminder, subset = country == "Afghanistan", type = "b")
plot(lifeExp ~ year, gapminder, subset = country == "United States", type = "b")

We can add a second variable to plot life expectancy and GDP

In [ ]:
plot(lifeExp ~ gdpPercap, gapminder, subset = year == 2007)

What if we change the X-axis scale to log?

In [ ]:
plot(lifeExp ~ gdpPercap, gapminder, subset = year == 2007, log = "x")

__CAUTION:__ Visualizing information in different ways can lead readers to different interpretations

Use boxplots to visualize grouped data

In [ ]:
boxplot(df$lifeExp ~ df$continent)
boxplot(df$gdpPercap ~ df$continent, log = "y")

In [ ]:
par(mfrow=c(1,2))
boxplot(df$lifeExp ~ df$continent)
boxplot(df$gdpPercap ~ df$continent, log = "y")

### Section II: Mapping with base plot

Base plotting is a simple less sophicated way to make maps in R

#### Step 1: Load a basemap from the _maptools_ package

In [ ]:
data("wrld_simpl")

#### Step 2: Plot the basemap

In [ ]:
plot(wrld_simpl)

#### Step 3: Define the extent manually using _xlim_ & _ylim_

In [ ]:
plot(wrld_simpl,
     xlim=c(-130,-60),
     ylim=c(25,50))

#### Step 4: Add colors to countries & background by using the _col_ & _bg_ functions

In [ ]:
plot(wrld_simpl,
     xlim=c(-130,-60),
     ylim=c(25,50),
     col='olivedrab3', #Countries
     bg='lightblue') #***Background in this case thats the ocean

#### Step 5: Add "UC" to our map by using defining the Lat and Long

In [ ]:
plot(wrld_simpl,
     xlim=c(-130,-60),
     ylim=c(25,50),
     col='olivedrab3', #Countries
     bg='lightblue') #***Background in this case thats the ocean

coords <- matrix(c(-84.518986, 39.132979),ncol=2)
coords <- coordinates(coords)
spoints <- SpatialPoints(coords)
df <- data.frame(location=c("UC"))
spointsdf <- SpatialPointsDataFrame(spoints,df)
plot(spointsdf,add=T,col=c('red'),pch=16) # add = T; adds this to existing plot

### Section III: Mapping with tmap

#### Step 1: Load the _world_ basemap

In [ ]:
world <- world

Explore the variables of the _world_ data

In [ ]:
names(world)

#### Step 2: Plot the _world_ data using  tmap


The tmap package uses a "layer cake" style programming to build visualizations. <br>
We start with a filled shape and then add elements to increase complexity

In [ ]:
tm_shape(world) +
  tm_fill()

In [ ]:
tm_shape(world) +
  tm_fill() + 
  tm_borders() 

#### Step 3: Create a thematic map using population density
Default style is pretty (rounded numbers)

In [ ]:
tm_shape(world) +
  tm_fill(col = "pop") + 
  tm_borders() 

#### Step 4: Explore thematic styles

1. define number of groups by using _n = _
2. Manually define breaks using _breaks = _
3. Or you can define a style such as _jenks_, _cont_, or _cat_

In [ ]:
tm_shape(world) +
  tm_fill(col = "pop", n =  5) + 
  tm_borders() 

In [ ]:
breaks = c(0,10^6, 10^7, 10^8, 10^9, 10^10)
tm_shape(world) +
  tm_fill(col = "pop", breaks = breaks) + 
  tm_borders() 

In [ ]:
tm_shape(world) +
  tm_fill(col = "pop", style = "jenks") + 
  tm_borders() 

In [ ]:
tm_shape(world) +
  tm_fill(col = "pop", style = "cont") + 
  tm_borders() 

In [ ]:
tm_shape(world) +
  tm_fill(col = "continent", style = "cat") + 
  tm_borders()

#### Step 5: Explore palattes

In [ ]:
#tmaptools::palette_explorer()
?palette

In [ ]:
tm_shape(world) +
  tm_fill(col = "pop", style = "cont", palette = "magma") + 
  tm_borders() 

#### Step 6: Add cartographic elements

In [ ]:
tm_shape(world) +
  tm_fill(col = "continent", style = "cat") + 
  tm_borders() +
  tm_layout(title = "World Countries by Continent",
            title.size = 2,
            frame = FALSE,
            inner.margins = 0.1,
            legend.title.size = 1,
            legend.text.size = 0.75,
            legend.outside = TRUE) +
  tm_compass(type = "arrow", position = c("right", "top"), size = 1) +
  tm_scale_bar(breaks = c(0, 5000, 10000),size = 0.5, position = c(0.6,0.075))


Mapping a globe color coded by continent isn't helpful, so lets use the same technique above to map life expectancy 

In [ ]:
tm_shape(world) +
  tm_fill(col = "lifeExp", style = "cont", colorNA = NULL) + 
  tm_borders() +
  tm_layout(title = "Life Expectancies by Country",
            title.size = 2,
            frame = FALSE,
            inner.margins = 0.1,
            legend.title.size = 1,
            legend.text.size = 0.75,
            legend.outside = TRUE) +
  tm_compass(type = "arrow", position = c("right", "top"), size = 1) +
  tm_scale_bar(breaks = c(0, 5000, 10000),size = 0.5, position = c(0.6,0.075))